In [1]:
!pip install nervaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from sklearn import preprocessing
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, LSTM, TimeDistributed, Bidirectional
from nervaluate import Evaluator
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
class txtReader:
    def __init__(self, filename):
        self.filename = filename

    def read_split(self):
        with open(self.filename, 'r') as f:
            file_read = f.read()
        lines = file_read.split('\n')
        text = []
        text_id = []
        tmp = []
        tmp_id = []
        for i in lines:
            if i == '':
                if len(tmp) != 0:
                    text.append(tmp)
                    text_id.append(tmp_id)
                    tmp = []
                    tmp_id = []
            else:
                word = i.split('\t')
                tmp.append(word[0])
                tmp_id.append(word[1])

        return text, text_id

# Nueva sección

In [4]:
class alphabet:
    def __init__(self, train_file, dev_file, test_file):
        self.train_file = train_file
        self.dev_file = dev_file
        self.test_file = test_file
        self.data = dict()
        self.labels = dict()

    def read_split(self):
        text_files = []
        for text_file in [self.train_file, self.dev_file, self.test_file]:
            txt = txtReader(text_file)
            text, text_id = txt.read_split()
            text_files.append(text)
            text_files.append(text_id)

        return text_files[0], text_files[1], text_files[2], text_files[3], text_files[4], text_files[5]
    
    def _tagger(self, dataset, cnt, dictionary):
        for i in dataset: # i es una frase
            for j in i: # j es una palabra
                pos = i.index(j) # pos es la posicion de la palabra en la frase
                if j not in dictionary:
                    dictionary[j] = cnt
                    i[pos] = cnt
                    cnt += 1
                else:
                    i[pos] = dictionary[j]

        return dataset, cnt, dictionary 

    def labelEncoder(self):
        train, train_id, dev, dev_id, test, test_id = self.read_split()
        cnt = 1
        cnt_id = 0

        train, cnt, self.data = self._tagger(train, cnt, self.data) 
        train_id, cnt_id, self.labels = self._tagger(train_id, cnt_id, self.labels)
        dev, cnt, self.data = self._tagger(dev, cnt, self.data)
        dev_id, cnt_id, self.labels = self._tagger(dev_id, cnt_id, self.labels)
        
        len_train = 0
        for i in train:
            len_train += len(i)

        for phrase_te in test:
            for te in phrase_te:
                pos = phrase_te.index(te)
                if te not in self.data:
                    self.data[te] = len_train
                    phrase_te[pos] = self.data[te]
                else:
                    phrase_te[pos] = self.data[te]

        for phrase_te_id in test_id:
            for te_id in phrase_te_id:
                pos_id = phrase_te_id.index(te_id)
                if te_id not in self.labels:
                    self.labels[te_id] = len_train
                    phrase_te_id[pos_id] = self.labels[te_id]
                else:
                    phrase_te_id[pos_id] = self.labels[te_id] 

        return train, train_id, dev, dev_id, test, test_id, self.data, self.labels

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_PartTUT, train_id_PartTUT, dev_PartTUT, dev_id_PartTUT, test_PartTUT, test_id_PartTUT, PartTUT_data, PartTUT_labels = alphabet('/content/drive/MyDrive/ple_assignment/datasets/PartTUT/train.txt', '/content/drive/MyDrive/ple_assignment/datasets/PartTUT/dev.txt', '/content/drive/MyDrive/ple_assignment/datasets/PartTUT/test.txt').labelEncoder()

In [ ]:
train_MITMovie, train_id_MITMovie, dev_MITMovie, dev_id_MITMovie, test_MITMovie, test_id_MITMovie, MITMovie_data, MITMovie_labels = alphabet('/content/drive/MyDrive/ple_assignment/datasets/MITMovie/train.txt', '/content/drive/MyDrive/ple_assignment/datasets/MITMovie/dev.txt', '/content/drive/MyDrive/ple_assignment/datasets/MITMovie/test.txt').labelEncoder()

In [6]:
train_MITRestaurant, train_id_MITRestaurant, dev_MITRestaurant, dev_id_MITRestaurant, test_MITRestaurant, test_id_MITRestaurant, MITRestaurant_data, MITRestaurant_labels = alphabet('/content/drive/MyDrive/ple_assignment/datasets/MITRestaurant/train.txt', '/content/drive/MyDrive/ple_assignment/datasets/MITRestaurant/dev.txt', '/content/drive/MyDrive/ple_assignment/datasets/MITRestaurant/test.txt').labelEncoder()

In [ ]:
# train_PartTUT, train_id_PartTUT, dev_PartTUT, dev_id_PartTUT, test_PartTUT, test_id_PartTUT, PartTUT_data, PartTUT_labels = alphabet('materiales_practica/datasets/PartTUT/train.txt', 'materiales_practica/datasets/PartTUT/dev.txt', 'materiales_practica/datasets/PartTUT/test.txt').labelEncoder()

In [ ]:
# train_MITMovie, train_id_MITMovie, dev_MITMovie, dev_id_MITMovie, test_MITMovie, test_id_MITMovie, MITMovie_data, MITMovie_labels = alphabet('materiales_practica/datasets/MITMovie/train.txt', 'materiales_practica/datasets/MITMovie/dev.txt', 'materiales_practica/datasets/MITMovie/test.txt').labelEncoder()

In [ ]:
# train_MITRestaurant, train_id_MITRestaurant, dev_MITRestaurant, dev_id_MITRestaurant, test_MITRestaurant, test_id_MITRestaurant, MITRestaurant_data, MITRestaurant_labels = alphabet('materiales_practica/datasets/MITRestaurant/train.txt', 'materiales_practica/datasets/MITRestaurant/dev.txt', 'materiales_practica/datasets/MITRestaurant/test.txt').labelEncoder()

In [25]:
class FFTagger():
    def __init__(self, train, train_id, dev, dev_id, test, test_id, labels_dict, n, loss, optimizer, metrics, batch_size, epochs): # weighted_metrics
        self.model = Sequential()
        self.train = train
        self.train_id = train_id
        self.dev = dev
        self.dev_id = dev_id
        self.test = test
        self.test_id = test_id
        self.labels_dict = labels_dict
        self.n = n
        len_train = 0
        for i in train:
            len_train += len(i)
        self.vocab_size = len_train + 1 # +1 por valores desconocidos en test
        self.classes = labels_dict.keys()
        self.num_classes = len(labels_dict)
        self.loss = loss
        self.optimizer = optimizer
        self.metrics = metrics
        # self.weighted_metrics = weighted_metrics
        self.train_windows = []
        self.dev_windows = []
        self.test_windows = []
        self.batch_size = batch_size
        self.epochs = epochs

    def build_model(self): 
        self.model.add(Input(shape=(self.n*2+1,), dtype=tf.int32))
        self.model.add(Embedding(input_dim = self.vocab_size, output_dim=20, mask_zero=True, input_length=self.n*2+1))
        self.model.add(Flatten())
        self.model.add(Dense(64, activation='relu'))
        self.model.add(Dense(self.num_classes, activation='softmax'))

    def train_model(self):
        # Añadimos padding a las frases y las dividimos en ventanas de tamaño 2n+1
        padding = []
        for i in range(self.n):
            padding.append(0)

        padded_train = []
        padded_dev = []
        one_hot_train_id = []
        one_hot_dev_id = []
        for j in range(len(self.train)):
            padded_phrase = padding + self.train[j] + padding
            padded_train.append(padded_phrase) # padding
            for w in range(self.n, len(padded_phrase)-self.n): # división en ventanas
                self.train_windows.append(padded_phrase[w-self.n:w+self.n+1])
            one_hot_phrase = to_categorical(self.train_id[j], num_classes=self.num_classes) # one-hot encoding
            for v in one_hot_phrase:
                one_hot_train_id.append(v)

        for k in range(len(self.dev)):
            padded_phrase_dev = padding + self.dev[k] + padding
            padded_dev.append(padded_phrase_dev)
            for w in range(self.n, len(padded_phrase_dev)-self.n):
                self.dev_windows.append(padded_phrase_dev[w-self.n:w+self.n+1])
            one_hot_phrase_dev = to_categorical(self.dev_id[k], num_classes=self.num_classes)
            for v in one_hot_phrase_dev:
                one_hot_dev_id.append(v)
        
        # Convertimos las listas en tensores
        train_tensor = tf.data.Dataset.from_tensor_slices((self.train_windows, one_hot_train_id))
        train_tensor = train_tensor.batch(self.batch_size)
        dev_tensor = tf.data.Dataset.from_tensor_slices((self.dev_windows, one_hot_dev_id))
        dev_tensor = dev_tensor.batch(self.batch_size)

        # Reescribimos las variables globales
        self.train = padded_train
        self.dev = padded_dev

        self.model.compile(loss=self.loss, optimizer=self.optimizer, metrics=self.metrics)
        self.model.fit(train_tensor, epochs=self.epochs, validation_data=dev_tensor, verbose=1)

    def evaluate_model(self, task):
        padding = []
        for i in range(self.n):
            padding.append(0)
        
        # Añadimos el padding, dividimos en ventanas y hacemos one-hot encoding de las etiquetas
        padded_test = []
        one_hot_test_id = []
        for j in range(len(self.test)):
            padded_phrase_test = padding + self.test[j] + padding
            padded_test.append(padded_phrase_test)
            for w in range(self.n, len(padded_phrase_test)-self.n): # división en ventanas
                self.test_windows.append(padded_phrase_test[w-self.n:w+self.n+1])
            one_hot_phrase_test = to_categorical(self.test_id[j], num_classes=self.num_classes) # one-hot encoding
            for v in one_hot_phrase_test:
                one_hot_test_id.append(v)

        test_labels = [] # guarda las etiquetas como cadena de caracteres
        test_labels_length = [] # guarda la longitud de las oraciones
        for i in self.test_id: 
            temp_phrase = []
            for j in i: 
                for k, v in self.labels_dict.items():
                    if j == v:
                        temp_phrase.append(k)
            test_labels.append(temp_phrase)
            test_labels_length.append(len(temp_phrase))

        test_tensor = tf.data.Dataset.from_tensor_slices((self.test_windows, one_hot_test_id))
        test_tensor = test_tensor.batch(self.batch_size)
        
        if task == "PoS":
            loss, accuracy = self.model.evaluate(test_tensor, verbose=1)
            return loss, accuracy
        elif task == "NER":
            loss, accuracy = self.model.evaluate(test_tensor, verbose=1)
            pred = self.model.predict(test_tensor).astype(np.float32)

            # para cada elemento de pred obtenemos la etiqueta numérica  
            predictions = np.argmax(pred, axis=-1)
            
            # convertimos las etiquetas numéricas a etiquetas de texto
            pred_labels = []
            test_index = 0
            tmp_phrase = []
            for l in predictions:
                for k, v in self.labels_dict.items():
                    if l == v:
                        tmp_phrase.append(k)
                    if len(tmp_phrase) == test_labels_length[test_index]:
                        pred_labels.append(tmp_phrase)
                        tmp_phrase = []
                        test_index += 1
                        break

            numeric_tags = set()
            for i in self.train_id:
                numeric_tags = set(numeric_tags|set(i))
            for j in self.dev_id:
                numeric_tags = set(numeric_tags|set(j))
            numeric_tags = list(numeric_tags)

            # convertimos las etiquetas numéricas a etiquetas de texto
            tags = []
            for t in numeric_tags:
                for k, v in self.labels_dict.items():
                    if t == v:
                        tags.append(k)

            # Nos quedamos con las etiquetas de la entidad (sin B e I)
            evaluator_tags = []
            for tag in tags:
                if tag == "O":
                    continue
                elif tag == '':
                    continue
                else:
                    evaluator_tag = tag[2:]

                if evaluator_tag not in evaluator_tags:
                    evaluator_tags.append(evaluator_tag)
            
            evaluator = Evaluator(test_labels, pred=pred_labels, tags=evaluator_tags, loader="list")
            results, results_by_tag = evaluator.evaluate()
            
            return loss, accuracy, results, results_by_tag
        else:
            return "Task not found"

In [ ]:
modelPartTUT = FFTagger(train_PartTUT, train_id_PartTUT, dev_PartTUT, dev_id_PartTUT, test_PartTUT, test_id_PartTUT,  PartTUT_labels, 2, 'categorical_crossentropy', 'adam', ['accuracy'], 32, 10)
modelPartTUT.build_model()
modelPartTUT.train_model()

Epoch 1/10
680/680 [==============================] - 16s 14ms/step - loss: 1.1979 - accuracy: 0.6753 - val_loss: 0.6194 - val_accuracy: 0.8051
Epoch 2/10
680/680 [==============================] - 3s 4ms/step - loss: 0.3281 - accuracy: 0.9106 - val_loss: 0.3897 - val_accuracy: 0.8761
Epoch 3/10
680/680 [==============================] - 3s 5ms/step - loss: 0.1396 - accuracy: 0.9637 - val_loss: 0.3520 - val_accuracy: 0.8876
Epoch 4/10
680/680 [==============================] - 2s 4ms/step - loss: 0.0796 - accuracy: 0.9793 - val_loss: 0.3470 - val_accuracy: 0.8929
Epoch 5/10
680/680 [==============================] - 2s 4ms/step - loss: 0.0512 - accuracy: 0.9863 - val_loss: 0.3565 - val_accuracy: 0.8936
Epoch 6/10
680/680 [==============================] - 2s 4ms/step - loss: 0.0335 - accuracy: 0.9913 - val_loss: 0.3764 - val_accuracy: 0.8940
Epoch 7/10
680/680 [==============================] - 3s 5ms/step - loss: 0.0218 - accuracy: 0.9946 - val_loss: 0.4001 - val_accuracy: 0.8936
Epoc

In [ ]:
modelPartTUT.evaluate_model("PoS")

53/53 [==============================] - 0s 3ms/step - loss: 0.3712 - accuracy: 0.9276


(0.3712385296821594, 0.9276179075241089)

In [ ]:
modelMITMovie = FFTagger(train_MITMovie, train_id_MITMovie, dev_MITMovie, dev_id_MITMovie, test_MITMovie, test_id_MITMovie, MITMovie_labels, 2, 'categorical_crossentropy', 'adam', ['accuracy'], 32, 10)
modelMITMovie.build_model()
modelMITMovie.train_model()

Epoch 1/10
2232/2232 [==============================] - 19s 8ms/step - loss: 0.8811 - accuracy: 0.7570 - val_loss: 0.5913 - val_accuracy: 0.8240
Epoch 2/10
2232/2232 [==============================] - 10s 5ms/step - loss: 0.4929 - accuracy: 0.8480 - val_loss: 0.5144 - val_accuracy: 0.8396
Epoch 3/10
2232/2232 [==============================] - 10s 5ms/step - loss: 0.3916 - accuracy: 0.8747 - val_loss: 0.5074 - val_accuracy: 0.8409
Epoch 4/10
2232/2232 [==============================] - 10s 4ms/step - loss: 0.3307 - accuracy: 0.8924 - val_loss: 0.5214 - val_accuracy: 0.8366
Epoch 5/10
2232/2232 [==============================] - 10s 5ms/step - loss: 0.2884 - accuracy: 0.9050 - val_loss: 0.5455 - val_accuracy: 0.8346
Epoch 6/10
2232/2232 [==============================] - 10s 5ms/step - loss: 0.2580 - accuracy: 0.9140 - val_loss: 0.5736 - val_accuracy: 0.8311
Epoch 7/10
2232/2232 [==============================] - 10s 5ms/step - loss: 0.2354 - accuracy: 0.9207 - val_loss: 0.6038 - val_ac

In [ ]:
modelMITMovie.evaluate_model("NER")

610/610 [==============================] - 1s 2ms/step


(0.7925033569335938,
 0.8087918758392334,
 {'ent_type': {'correct': 4929,
   'incorrect': 514,
   'partial': 0,
   'missed': 243,
   'spurious': 1991,
   'possible': 5686,
   'actual': 7434,
   'precision': 0.6630347054075868,
   'recall': 0.8668659866338375,
   'f1': 0.7513719512195123},
  'partial': {'correct': 3574,
   'incorrect': 0,
   'partial': 1869,
   'missed': 243,
   'spurious': 1991,
   'possible': 5686,
   'actual': 7434,
   'precision': 0.606470271724509,
   'recall': 0.7929124164614844,
   'f1': 0.6872713414634146},
  'strict': {'correct': 3419,
   'incorrect': 2024,
   'partial': 0,
   'missed': 243,
   'spurious': 1991,
   'possible': 5686,
   'actual': 7434,
   'precision': 0.45991390906645147,
   'recall': 0.601301442138586,
   'f1': 0.521189024390244},
  'exact': {'correct': 3574,
   'incorrect': 1869,
   'partial': 0,
   'missed': 243,
   'spurious': 1991,
   'possible': 5686,
   'actual': 7434,
   'precision': 0.4807640570352435,
   'recall': 0.6285613788251847,
 

In [26]:
modelMITRestaurant = FFTagger(train_MITRestaurant, train_id_MITRestaurant, dev_MITRestaurant, dev_id_MITRestaurant, test_MITRestaurant, test_id_MITRestaurant, MITRestaurant_labels, 2, 'categorical_crossentropy', 'adam', ['accuracy'], 32, 5)
modelMITRestaurant.build_model()
modelMITRestaurant.train_model()

Epoch 1/5
1986/1986 [==============================] - 16s 7ms/step - loss: 0.8160 - accuracy: 0.7756 - val_loss: 0.4825 - val_accuracy: 0.8632
Epoch 2/5
1986/1986 [==============================] - 8s 4ms/step - loss: 0.3660 - accuracy: 0.8945 - val_loss: 0.3929 - val_accuracy: 0.8832
Epoch 3/5
1986/1986 [==============================] - 8s 4ms/step - loss: 0.2707 - accuracy: 0.9191 - val_loss: 0.3844 - val_accuracy: 0.8865
Epoch 4/5
1986/1986 [==============================] - 7s 4ms/step - loss: 0.2226 - accuracy: 0.9318 - val_loss: 0.3990 - val_accuracy: 0.8857
Epoch 5/5
1986/1986 [==============================] - 9s 4ms/step - loss: 0.1926 - accuracy: 0.9410 - val_loss: 0.4203 - val_accuracy: 0.8823


In [27]:
modelMITRestaurant.evaluate_model("NER")

446/446 [==============================] - 1s 1ms/step


(0.42892056703567505,
 0.8813915848731995,
 {'ent_type': {'correct': 2725,
   'incorrect': 266,
   'partial': 0,
   'missed': 160,
   'spurious': 427,
   'possible': 3151,
   'actual': 3418,
   'precision': 0.7972498537156232,
   'recall': 0.8648048238654396,
   'f1': 0.8296544375095143},
  'partial': {'correct': 2448,
   'incorrect': 0,
   'partial': 543,
   'missed': 160,
   'spurious': 427,
   'possible': 3151,
   'actual': 3418,
   'precision': 0.795640725570509,
   'recall': 0.8630593462392891,
   'f1': 0.8279799056172933},
  'strict': {'correct': 2307,
   'incorrect': 684,
   'partial': 0,
   'missed': 160,
   'spurious': 427,
   'possible': 3151,
   'actual': 3418,
   'precision': 0.6749561146869514,
   'recall': 0.732148524278007,
   'f1': 0.7023900137007155},
  'exact': {'correct': 2448,
   'incorrect': 543,
   'partial': 0,
   'missed': 160,
   'spurious': 427,
   'possible': 3151,
   'actual': 3418,
   'precision': 0.7162083089526039,
   'recall': 0.7768962234211362,
   'f1'

In [22]:
class LSTMTagger():
    def __init__(self, train, train_id, dev, dev_id, test, test_id, data_dict, labels_dict, loss, optimizer, metrics, batch_size, epochs):
        self.model = Sequential()
        self.train = train
        self.train_id = train_id
        self.dev = dev
        self.dev_id = dev_id
        self.test = test
        self.test_id = test_id
        self.data_dict = data_dict
        self.labels_dict = labels_dict
        len_train = 0
        for i in train:
            len_train += len(i)
        self.vocab_size = len_train + 1
        self.classes = labels_dict.keys()
        self.num_classes = len(labels_dict)
        self.loss=loss
        self.optimizer=optimizer
        self.metrics=metrics
        self.train_windows = []
        self.train_windows_id = []
        self.dev_windows = []
        self.dev_windows_id = []
        self.test_windows = []
        self.test_windows_id = []
        self.maxlen = 0
        self.batch_size = batch_size
        self.epochs = epochs

    def build_model(self, bidirectional=False):
        self.model.add(Embedding(input_dim=self.vocab_size, output_dim=20, mask_zero=True, input_length=self.maxlen))
        if bidirectional:
            self.model.add(Bidirectional(LSTM(64, return_sequences=True)))
        else:
            self.model.add(LSTM(64, return_sequences=True))
        self.model.add(TimeDistributed(Dense(units=self.num_classes, activation='softmax'))) 

    def _getMaxLength(self, data):
        for i in range(len(data)):
            if len(data[i]) > self.maxlen:
                self.maxlen = len(data[i])
        
    def _padder(self, data, data_id):
        padded_data = pad_sequences(data, maxlen=self.maxlen, padding='post')
        padded_data_id = pad_sequences(data_id, maxlen=self.maxlen, padding='post')
        one_hot_data_id = to_categorical(padded_data_id, num_classes=self.num_classes)

        return padded_data, one_hot_data_id
          

    def preprocessing(self):
        self._getMaxLength(self.train)
        self._getMaxLength(self.dev)
        self._getMaxLength(self.test)

        datasets = []
        for data in [[self.train, self.train_id], [self.dev, self.dev_id], [self.test, self.test_id]]:
          padded_data = pad_sequences(data[0], maxlen=self.maxlen, padding='post')
          padded_data_id = pad_sequences(data[1], maxlen=self.maxlen, padding='post')
          one_hot_data_id = to_categorical(padded_data_id, num_classes=self.num_classes)
          datasets.append([padded_data, one_hot_data_id])

        self.train_windows, self.train_windows_id = datasets[0]
        self.dev_windows, self.dev_windows_id = datasets[1]
        self.test_windows, self.test_windows_id = datasets[2]

    def train_model(self):
        train_tensor = tf.data.Dataset.from_tensor_slices((self.train_windows, self.train_windows_id))
        train_tensor = train_tensor.batch(self.batch_size)
        dev_tensor = tf.data.Dataset.from_tensor_slices((self.dev_windows, self.dev_windows_id))
        dev_tensor = dev_tensor.batch(self.batch_size)

        self.model.compile(loss=self.loss, optimizer=self.optimizer, metrics=self.metrics)
        self.model.fit(train_tensor, epochs=self.epochs, validation_data=dev_tensor)

    def evaluate_model(self, task):
        test_labels = [] # lista que almacena las etiquetas de los elementos de self.test_id
        # Cambiamos las etiquetas de numérico a string
        for i in self.test_id:
            temp_phrase = []
            for j in i:
                for k, v in self.labels_dict.items():
                    if j == v:
                        temp_phrase.append(k)
            test_labels.append(temp_phrase)
        
        lengths = []
        for t in self.test_id:
            lengths.append(len(t))

        test_tensor = tf.data.Dataset.from_tensor_slices((self.test_windows, self.test_windows_id))
        test_tensor = test_tensor.batch(self.batch_size)
        
        if task == "PoS":
            loss, accuracy = self.model.evaluate(test_tensor, verbose=1)
            return loss, accuracy
        elif task == "NER":
            loss, accuracy = self.model.evaluate(test_tensor, verbose=1)
            pred = self.model.predict(test_tensor).astype(np.float32)

            # convertimos las predicciones a etiquetas numéricas
            pred_ids = np.argmax(pred, axis=-1)

            pred_wo_padding = []
            for p in range(len(pred_ids)):
                index = lengths[p]
                pred_wo_padding.append(pred_ids[p][:index])

            # convertimos las etiquetas numéricas a etiquetas de texto
            pred_labels = []
            for l in pred_wo_padding:
                tmp_phrase = []
                for w in l:
                    for k, v in self.labels_dict.items():
                        if w == v:
                            tmp_phrase.append(k)
                pred_labels.append(tmp_phrase)
            
            # obtenemos las etiquetas únicas que hay en el conjunto de train y dev
            numeric_tags = set()
            for i in self.train_id:
                numeric_tags = set(numeric_tags|set(i))
            for j in self.dev_id:
                numeric_tags = set(numeric_tags|set(j))
            numeric_tags = list(numeric_tags)

            # convertimos las etiquetas numéricas a etiquetas de texto
            tags = []
            for t in numeric_tags:
                for k, v in self.labels_dict.items():
                    if t == v:
                        tags.append(k)

            # Nos quedamos con las etiquetas de cada entidad
            evaluator_tags = []
            for tag in tags:
                if tag == "O":
                    continue
                elif tag == '':
                    continue
                else:
                    evaluator_tag = tag[2:]

                if evaluator_tag not in evaluator_tags:
                    evaluator_tags.append(evaluator_tag)

            print(pred_labels[0:10])

            evaluator = Evaluator(test_labels, pred=pred_labels, tags=evaluator_tags, loader="list")
            results, results_by_tag = evaluator.evaluate()
            
            return loss, accuracy, results, results_by_tag
        else:
            return "Task not found"

In [ ]:
BDLSTMPartut = LSTMTagger(train_PartTUT, train_id_PartTUT, dev_PartTUT, dev_id_PartTUT, test_PartTUT, test_id_PartTUT, PartTUT_data, PartTUT_labels, 'categorical_crossentropy', 'adam', ['accuracy'], 32, 50)
BDLSTMPartut.preprocessing()
BDLSTMPartut.build_model(bidirectional=True)
BDLSTMPartut.train_model()

Epoch 1/50
49/56 [=========================>....] - ETA: 1s - loss: 2.5837 - accuracy: 0.2118

KeyboardInterrupt: ignored

In [ ]:
BDLSTMPartut.evaluate_model("PoS")

5/5 [==============================] - 0s 12ms/step - loss: 0.3776 - accuracy: 0.9232


(0.37755945324897766, 0.9231681823730469)

In [ ]:
LSTMPartut = LSTMTagger(train_PartTUT, train_id_PartTUT, dev_PartTUT, dev_id_PartTUT, test_PartTUT, test_id_PartTUT, PartTUT_data, PartTUT_labels, 'categorical_crossentropy', 'adam', ['accuracy'], 32, 50)
LSTMPartut.preprocessing()
LSTMPartut.build_model(bidirectional=False)
LSTMPartut.train_model()

Epoch 1/50
56/56 [==============================] - 17s 222ms/step - loss: 2.5943 - accuracy: 0.2073 - val_loss: 2.4684 - val_accuracy: 0.2087
Epoch 2/50
56/56 [==============================] - 1s 11ms/step - loss: 2.3899 - accuracy: 0.2431 - val_loss: 2.3094 - val_accuracy: 0.3285
Epoch 3/50
56/56 [==============================] - 1s 11ms/step - loss: 2.1140 - accuracy: 0.3803 - val_loss: 1.8978 - val_accuracy: 0.5205
Epoch 4/50
56/56 [==============================] - 1s 11ms/step - loss: 1.6081 - accuracy: 0.5720 - val_loss: 1.4614 - val_accuracy: 0.5993
Epoch 5/50
56/56 [==============================] - 1s 11ms/step - loss: 1.2013 - accuracy: 0.6669 - val_loss: 1.1895 - val_accuracy: 0.6629
Epoch 6/50
56/56 [==============================] - 1s 11ms/step - loss: 0.9262 - accuracy: 0.7441 - val_loss: 0.9960 - val_accuracy: 0.7455
Epoch 7/50
56/56 [==============================] - 1s 11ms/step - loss: 0.7147 - accuracy: 0.8229 - val_loss: 0.8519 - val_accuracy: 0.7876
Epoch 8/50


In [ ]:
LSTMPartut.evaluate_model("PoS")

3/3 [==============================] - 0s 8ms/step - loss: 0.8699 - accuracy: 0.8004


(0.8698592185974121, 0.8003559708595276)

In [ ]:
BDLSTMMITMovie = LSTMTagger(train_MITMovie, train_id_MITMovie, dev_MITMovie, dev_id_MITMovie, test_MITMovie, test_id_MITMovie, MITMovie_data, MITMovie_labels, 'categorical_crossentropy', 'adam', ['accuracy'], 32, 1)
BDLSTMMITMovie.preprocessing()
BDLSTMMITMovie.build_model(bidirectional=True)
BDLSTMMITMovie.train_model()

220/220 [==============================] - 54s 209ms/step - loss: 1.7416 - accuracy: 0.5752 - val_loss: 1.3866 - val_accuracy: 0.6359


In [ ]:
BDLSTMMITMovie.evaluate_model("NER")

62/62 [==============================] - 2s 4ms/step
[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot'], ['O', 'O', 'O', 'O', 'O', 'O'

(1.460282325744629,
 0.6166102886199951,
 {'ent_type': {'correct': 1281,
   'incorrect': 597,
   'partial': 0,
   'missed': 3808,
   'spurious': 165,
   'possible': 5686,
   'actual': 2043,
   'precision': 0.6270190895741556,
   'recall': 0.22529018642279283,
   'f1': 0.33147884590503296},
  'partial': {'correct': 280,
   'incorrect': 0,
   'partial': 1598,
   'missed': 3808,
   'spurious': 165,
   'possible': 5686,
   'actual': 2043,
   'precision': 0.5281448849730788,
   'recall': 0.18976433345058039,
   'f1': 0.27920817699573036},
  'strict': {'correct': 227,
   'incorrect': 1651,
   'partial': 0,
   'missed': 3808,
   'spurious': 165,
   'possible': 5686,
   'actual': 2043,
   'precision': 0.1111111111111111,
   'recall': 0.039922616953921915,
   'f1': 0.058739811101048006},
  'exact': {'correct': 280,
   'incorrect': 1598,
   'partial': 0,
   'missed': 3808,
   'spurious': 165,
   'possible': 5686,
   'actual': 2043,
   'precision': 0.13705335291238374,
   'recall': 0.049243756595

In [ ]:
LSTMMITMovie = LSTMTagger(train_MITMovie, train_id_MITMovie, dev_MITMovie, dev_id_MITMovie, test_MITMovie, test_id_MITMovie, MITMovie_data, MITMovie_labels, 'categorical_crossentropy', 'adam', ['accuracy'], 32, 5)
LSTMMITMovie.preprocessing()
LSTMMITMovie.build_model(bidirectional=False)
LSTMMITMovie.train_model()

Epoch 1/5
220/220 [==============================] - 33s 94ms/step - loss: 1.9121 - accuracy: 0.5495 - val_loss: 1.4537 - val_accuracy: 0.6252
Epoch 2/5
220/220 [==============================] - 2s 9ms/step - loss: 1.2472 - accuracy: 0.6607 - val_loss: 1.0513 - val_accuracy: 0.7111
Epoch 3/5
220/220 [==============================] - 2s 11ms/step - loss: 0.9513 - accuracy: 0.7357 - val_loss: 0.8722 - val_accuracy: 0.7480
Epoch 4/5
220/220 [==============================] - 3s 13ms/step - loss: 0.7771 - accuracy: 0.7818 - val_loss: 0.7577 - val_accuracy: 0.7808
Epoch 5/5
220/220 [==============================] - 2s 9ms/step - loss: 0.6560 - accuracy: 0.8164 - val_loss: 0.7007 - val_accuracy: 0.8013


In [ ]:
LSTMMITMovie.evaluate_model("NER")

62/62 [==============================] - 1s 3ms/step
[['O', 'O', 'O', 'O', 'O', 'B-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot'], ['O', 'B-Year', 'B-Genre', 'I-Genre', 'I-Genre', 'O', 'B-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'O', 'O', 'B-Director', 'I-Actor', 'O', 'B-Actor', 'I-Actor'], ['O', 'O', 'B-Genre', 'I-Genre', 'O', 'B-Director', 'I-Director', 'O', 'I-Origin', 'O', 'O', 'O', 'I-Origin', 'I-Origin', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'B-Actor', 'I-Actor', 'O', 'B-Actor', 'O', 'B-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot'], ['O', 'O', 'O', 'B-Year', 'I-Director', 'O', 'B-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot', 'I-Plot'], ['O', 'O', 'B-Genre', 'I-Genre', 'I-Genre',

(0.7579349875450134,
 0.7873757481575012,
 {'ent_type': {'correct': 4031,
   'incorrect': 1172,
   'partial': 0,
   'missed': 483,
   'spurious': 1335,
   'possible': 5686,
   'actual': 6538,
   'precision': 0.616549403487305,
   'recall': 0.7089342244108336,
   'f1': 0.6595222513089006},
  'partial': {'correct': 3270,
   'incorrect': 0,
   'partial': 1933,
   'missed': 483,
   'spurious': 1335,
   'possible': 5686,
   'actual': 6538,
   'precision': 0.647981033955338,
   'recall': 0.7450756243404854,
   'f1': 0.6931446335078535},
  'strict': {'correct': 2775,
   'incorrect': 2428,
   'partial': 0,
   'missed': 483,
   'spurious': 1335,
   'possible': 5686,
   'actual': 6538,
   'precision': 0.42444172529825636,
   'recall': 0.48804080196975025,
   'f1': 0.4540248691099476},
  'exact': {'correct': 3270,
   'incorrect': 1933,
   'partial': 0,
   'missed': 483,
   'spurious': 1335,
   'possible': 5686,
   'actual': 6538,
   'precision': 0.5001529519730804,
   'recall': 0.5750967288075977

In [23]:
BDLSTMMITRestaurant = LSTMTagger(train_MITRestaurant, train_id_MITRestaurant, dev_MITRestaurant, dev_id_MITRestaurant, test_MITRestaurant, test_id_MITRestaurant, MITRestaurant_data, MITRestaurant_labels, 'categorical_crossentropy', 'adam', ['accuracy'], 32, 1)
BDLSTMMITRestaurant.preprocessing()
BDLSTMMITRestaurant.build_model(bidirectional=True)
BDLSTMMITRestaurant.train_model()

216/216 [==============================] - 32s 103ms/step - loss: 1.7218 - accuracy: 0.6200 - val_loss: 1.4037 - val_accuracy: 0.6423


In [24]:
BDLSTMMITRestaurant.evaluate_model("NER")

48/48 [==============================] - 3s 6ms/step
[['O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Location', 'I-Location'], ['O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Location']]


(1.4425374269485474,
 0.6248158812522888,
 {'ent_type': {'correct': 266,
   'incorrect': 487,
   'partial': 0,
   'missed': 2398,
   'spurious': 58,
   'possible': 3151,
   'actual': 811,
   'precision': 0.3279901356350185,
   'recall': 0.08441764519200254,
   'f1': 0.13427561837455831},
  'partial': {'correct': 249,
   'incorrect': 0,
   'partial': 504,
   'missed': 2398,
   'spurious': 58,
   'possible': 3151,
   'actual': 811,
   'precision': 0.6177558569667078,
   'recall': 0.1589971437638845,
   'f1': 0.25290257445734476},
  'strict': {'correct': 88,
   'incorrect': 665,
   'partial': 0,
   'missed': 2398,
   'spurious': 58,
   'possible': 3151,
   'actual': 811,
   'precision': 0.10850801479654747,
   'recall': 0.027927642018406855,
   'f1': 0.04442200908632005},
  'exact': {'correct': 249,
   'incorrect': 504,
   'partial': 0,
   'missed': 2398,
   'spurious': 58,
   'possible': 3151,
   'actual': 811,
   'precision': 0.3070283600493218,
   'recall': 0.07902253252935576,
   'f1'

In [ ]:
LSTMMITRestaurant = LSTMTagger(train_MITRestaurant, train_id_MITRestaurant, dev_MITRestaurant, dev_id_MITRestaurant, test_MITRestaurant, test_id_MITRestaurant, MITRestaurant_data, MITRestaurant_labels, 'categorical_crossentropy', 'adam', ['accuracy'], 32, 50)
LSTMMITRestaurant.preprocessing()
LSTMMITRestaurant.build_model(bidirectional=False)
LSTMMITRestaurant.train_model()

Epoch 1/50
216/216 [==============================] - 25s 96ms/step - loss: 1.8936 - accuracy: 0.6192 - val_loss: 1.3114 - val_accuracy: 0.6575
Epoch 2/50
216/216 [==============================] - 2s 9ms/step - loss: 1.1864 - accuracy: 0.6665 - val_loss: 1.0818 - val_accuracy: 0.6771
Epoch 3/50
216/216 [==============================] - 2s 9ms/step - loss: 1.0143 - accuracy: 0.7087 - val_loss: 0.9581 - val_accuracy: 0.7191
Epoch 4/50
216/216 [==============================] - 3s 15ms/step - loss: 0.8494 - accuracy: 0.7617 - val_loss: 0.7949 - val_accuracy: 0.7811
Epoch 5/50
216/216 [==============================] - 2s 11ms/step - loss: 0.7051 - accuracy: 0.8043 - val_loss: 0.6904 - val_accuracy: 0.8020
Epoch 6/50
216/216 [==============================] - 2s 9ms/step - loss: 0.5993 - accuracy: 0.8374 - val_loss: 0.6146 - val_accuracy: 0.8225
Epoch 7/50
216/216 [==============================] - 2s 9ms/step - loss: 0.5099 - accuracy: 0.8619 - val_loss: 0.5657 - val_accuracy: 0.8364
Ep

In [ ]:
LSTMMITRestaurant.evaluate_model("NER")

48/48 [==============================] - 1s 3ms/step


(0.7898380756378174,
 0.8426627516746521,
 {'ent_type': {'correct': 0,
   'incorrect': 0,
   'partial': 0,
   'missed': 3151,
   'spurious': 0,
   'possible': 3151,
   'actual': 0,
   'precision': 0,
   'recall': 0.0,
   'f1': 0},
  'partial': {'correct': 0,
   'incorrect': 0,
   'partial': 0,
   'missed': 3151,
   'spurious': 0,
   'possible': 3151,
   'actual': 0,
   'precision': 0,
   'recall': 0.0,
   'f1': 0},
  'strict': {'correct': 0,
   'incorrect': 0,
   'partial': 0,
   'missed': 3151,
   'spurious': 0,
   'possible': 3151,
   'actual': 0,
   'precision': 0,
   'recall': 0.0,
   'f1': 0},
  'exact': {'correct': 0,
   'incorrect': 0,
   'partial': 0,
   'missed': 3151,
   'spurious': 0,
   'possible': 3151,
   'actual': 0,
   'precision': 0,
   'recall': 0.0,
   'f1': 0}},
 {'Rating': {'ent_type': {'correct': 0,
    'incorrect': 0,
    'partial': 0,
    'missed': 201,
    'spurious': 0,
    'possible': 201,
    'actual': 0,
    'precision': 0,
    'recall': 0.0,
    'f1': 0},
